In [1]:
# Import modules
import sys
import os
import pandas as pd

In [2]:
# Append the project root path to sys.path
sys.path.append(os.path.abspath(".."))

In [3]:
%load_ext autoreload
%autoreload 2

In [14]:
# Now import the modules
from src.data_cleaning import load_csv, clean_dataframe, save_cleaned_data
from src.DB_setup import get_db_connection, create_table, insert_data

In [10]:
df = load_csv("../data/telegram_messages.csv")

# Show first few rows
df.head(10)

2025-02-03 21:01:09,513 - INFO -  CSV file '../data/telegram_messages.csv' loaded successfully.


,channel_title,channel_username,message_id,message_text,date,media_path
0,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15271,PEDIASURE 3+ 1.6KG\nPrice 6000 birr \nTelegram...,2025-02-03 14:57:15+00:00,../data/photos\lobelia4cosmetics-15271.jpg
1,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15270,APTAMIL NO.3\nPrice 4000 birr \nTelegram @Lobe...,2025-02-03 13:49:39+00:00,../data/photos\lobelia4cosmetics-15270.jpg
2,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15269,APTAMIL NO.2\nPrice 4000 birr \nTelegram @Lobe...,2025-02-03 13:49:39+00:00,../data/photos\lobelia4cosmetics-15269.jpg
3,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15268,MULTI FOR HIM 90 TABLETS \nPrice 5000 birr \nT...,2025-02-03 13:35:38+00:00,../data/photos\lobelia4cosmetics-15268.jpg
4,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15267,POSTNATAL 60 SOFTGELS \nPrice 6500 birr \nTele...,2025-02-03 13:17:53+00:00,../data/photos\lobelia4cosmetics-15267.jpg
5,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15266,MULTI FOR HER 90 TABLETS \nPrice 5000 birr \nT...,2025-02-03 13:17:53+00:00,../data/photos\lobelia4cosmetics-15266.jpg
6,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15265,NaN,2025-02-03 04:25:31+00:00,../data/photos\lobelia4cosmetics-15265.jpg
7,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15264,NaN,2025-02-03 04:25:31+00:00,../data/photos\lobelia4cosmetics-15264.jpg
8,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15263,EOS LOTION \nPrice 3800 birr \nTelegram @Lobel...,2025-02-03 04:25:31+00:00,../data/photos\lobelia4cosmetics-15263.jpg
9,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15262,BATANA OIL 150ML\nPrice 4500 birr \nTelegram @...,2025-02-03 04:25:31+00:00,../data/photos\lobelia4cosmetics-15262.jpg


In [11]:
df_cleaned = clean_dataframe(df)

# Display cleaned dataset
df_cleaned.head()

2025-02-03 21:01:14,127 - INFO -  Date column formatted to datetime.
2025-02-03 21:01:14,127 - INFO -  Missing values filled.
2025-02-03 21:01:14,127 - INFO -  Text columns standardized.
2025-02-03 21:01:14,146 - INFO -  Data cleaning completed successfully.


,channel_title,channel_username,message_id,message,message_date,media_path
0,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15271,PEDIASURE 3+ 1.6KG Price 6000 birr Telegram @...,2025-02-03 14:57:15+00:00,../data/photos\lobelia4cosmetics-15271.jpg
1,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15270,APTAMIL NO.3 Price 4000 birr Telegram @Lobeli...,2025-02-03 13:49:39+00:00,../data/photos\lobelia4cosmetics-15270.jpg
2,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15269,APTAMIL NO.2 Price 4000 birr Telegram @Lobeli...,2025-02-03 13:49:39+00:00,../data/photos\lobelia4cosmetics-15269.jpg
3,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15268,MULTI FOR HIM 90 TABLETS Price 5000 birr Tel...,2025-02-03 13:35:38+00:00,../data/photos\lobelia4cosmetics-15268.jpg
4,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15267,POSTNATAL 60 SOFTGELS Price 6500 birr Telegr...,2025-02-03 13:17:53+00:00,../data/photos\lobelia4cosmetics-15267.jpg


In [12]:
# Check for missing values in the cleaned DataFrame
missing_values = df_cleaned.isnull().sum()
missing_values[missing_values > 0]  # Display only columns with missing values

Series([], dtype: int64)

In [19]:
# Drop the row with missing message_date
cleaned_df = df_cleaned.dropna(subset=['message_date'])

In [ ]:
# Check for missing values in the cleaned DataFrame
missing_values = cleaned_df.isnull().sum()
missing_values[missing_values > 0]  # Display only columns with missing values


In [13]:
# Save cleaned data to CSV
output_path = "../data/cleaned_telegram_data.csv"
save_cleaned_data(df_cleaned, output_path)


2025-02-03 21:01:23,828 - INFO -  Cleaned data saved successfully to '../data/cleaned_telegram_data.csv'.


 Cleaned data saved successfully to '../data/cleaned_telegram_data.csv'.


In [17]:
engine = get_db_connection()

2025-02-03 21:10:37,076 - INFO -  Successfully connected to the PostgreSQL database.


In [18]:
create_table(engine)

2025-02-03 21:10:40,504 - INFO -  Table 'telegram_messages' created successfully.


In [19]:
# Load the cleaned CSV into a DataFrame
cleaned_df = pd.read_csv("../data/cleaned_telegram_data.csv")

In [20]:
# Ensure the 'message_date' column is in datetime format (to prevent NaT issues)
cleaned_df["message_date"] = pd.to_datetime(cleaned_df["message_date"], errors="coerce")

# Check if there are any missing values before inserting
missing_values = cleaned_df.isnull().sum()
print("Missing Values Before Insert:", missing_values)

Missing Values Before Insert: channel_title       0
channel_username    0
message_id          0
message             0
message_date        0
media_path          0
dtype: int64


In [21]:
df_cleaned

,channel_title,channel_username,message_id,message,message_date,media_path
0,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15271,PEDIASURE 3+ 1.6KG Price 6000 birr Telegram @...,2025-02-03 14:57:15+00:00,../data/photos\lobelia4cosmetics-15271.jpg
1,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15270,APTAMIL NO.3 Price 4000 birr Telegram @Lobeli...,2025-02-03 13:49:39+00:00,../data/photos\lobelia4cosmetics-15270.jpg
2,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15269,APTAMIL NO.2 Price 4000 birr Telegram @Lobeli...,2025-02-03 13:49:39+00:00,../data/photos\lobelia4cosmetics-15269.jpg
3,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15268,MULTI FOR HIM 90 TABLETS Price 5000 birr Tel...,2025-02-03 13:35:38+00:00,../data/photos\lobelia4cosmetics-15268.jpg
4,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15267,POSTNATAL 60 SOFTGELS Price 6500 birr Telegr...,2025-02-03 13:17:53+00:00,../data/photos\lobelia4cosmetics-15267.jpg
...,...,...,...,...,...,...
145,Doctors Ethiopia,@DoctorsET,805,ሽንቶን ሲሸኑ ምን አይነት ቀለም አለው ? ሽንት ላይ ደም አለ ሲባል ምን...,2022-04-03 16:15:29+00:00,../data/photos\DoctorsET-805.jpg
146,Doctors Ethiopia,@DoctorsET,804,ላለፉት 21 አመታት በመምህርነት በተለያዩ ትምህርት ቤቶች ውስጥ አስተምሬ...,2022-04-02 06:06:36+00:00,../data/photos\DoctorsET-804.jpg
147,Doctors Ethiopia,@DoctorsET,802,No Message,2022-04-01 17:02:54+00:00,../data/photos\DoctorsET-802.jpg
148,Doctors Ethiopia,@DoctorsET,801,No Message,2022-04-01 17:02:54+00:00,../data/photos\DoctorsET-801.jpg


In [23]:
# Insert into the database
insert_data(engine, cleaned_df)

2025-02-03 21:13:15,129 - INFO - Inserting: 15271 - 2025-02-03 14:57:15+00:00
2025-02-03 21:13:15,136 - INFO - Inserting: 15270 - 2025-02-03 13:49:39+00:00
2025-02-03 21:13:15,138 - INFO - Inserting: 15269 - 2025-02-03 13:49:39+00:00
2025-02-03 21:13:15,140 - INFO - Inserting: 15268 - 2025-02-03 13:35:38+00:00
2025-02-03 21:13:15,142 - INFO - Inserting: 15267 - 2025-02-03 13:17:53+00:00
2025-02-03 21:13:15,144 - INFO - Inserting: 15266 - 2025-02-03 13:17:53+00:00
2025-02-03 21:13:15,144 - INFO - Inserting: 15265 - 2025-02-03 04:25:31+00:00
2025-02-03 21:13:15,151 - INFO - Inserting: 15264 - 2025-02-03 04:25:31+00:00
2025-02-03 21:13:15,153 - INFO - Inserting: 15263 - 2025-02-03 04:25:31+00:00
2025-02-03 21:13:15,154 - INFO - Inserting: 15262 - 2025-02-03 04:25:31+00:00
2025-02-03 21:13:15,156 - INFO - Inserting: 15261 - 2025-02-03 04:25:31+00:00
2025-02-03 21:13:15,159 - INFO - Inserting: 15260 - 2025-02-03 04:25:31+00:00
2025-02-03 21:13:15,161 - INFO - Inserting: 15259 - 2025-02-03 0

In [24]:
query = "SELECT * FROM telegram_messages LIMIT 5;"
df_pg = pd.read_sql(query, engine)

df_pg


,id,channel_title,channel_username,message_id,message,message_date,media_path
0,1,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15271,PEDIASURE 3+ 1.6KG Price 6000 birr Telegram @...,2025-02-03 14:57:15,../data/photos\lobelia4cosmetics-15271.jpg
1,2,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15270,APTAMIL NO.3 Price 4000 birr Telegram @Lobeli...,2025-02-03 13:49:39,../data/photos\lobelia4cosmetics-15270.jpg
2,3,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15269,APTAMIL NO.2 Price 4000 birr Telegram @Lobeli...,2025-02-03 13:49:39,../data/photos\lobelia4cosmetics-15269.jpg
3,4,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15268,MULTI FOR HIM 90 TABLETS Price 5000 birr Tel...,2025-02-03 13:35:38,../data/photos\lobelia4cosmetics-15268.jpg
4,5,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15267,POSTNATAL 60 SOFTGELS Price 6500 birr Telegr...,2025-02-03 13:17:53,../data/photos\lobelia4cosmetics-15267.jpg
